In [1]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
import random

In [2]:
class TreeCell(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.burning = False

    def step(self):
        if self.burning:
            self.burning = False

In [3]:
class Firefighter(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        possible_moves = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)

        cellmates = self.model.grid.get_cell_list_contents([new_position])
        for cellmate in cellmates:
            if isinstance(cellmate, TreeCell) and cellmate.burning:
                cellmate.burning = False

In [4]:
class FireModel(Model):
    def __init__(self, width, height, density):
        super().__init__()
        self.num_agents = 5
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Create tree cells
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                if random.random() < density:
                    tree = TreeCell((x, y), self)
                    self.grid.place_agent(tree, (x, y))
                    if random.random() < 0.25:
                        tree.burning = True

        for i in range(self.num_agents):
            firefighter = Firefighter(i, self)
            self.schedule.add(firefighter)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(firefighter, (x, y))

    def step(self):
        self.schedule.step()

In [5]:
def agent_portrayal(agent):
    if isinstance(agent, TreeCell):
        portrayal = {"Shape": "rect", "w": 1, "h": 1, "Filled": "true"}
        if agent.burning:
            portrayal["Color"] = "red"
        else:
            portrayal["Color"] = "green"
        portrayal["Layer"] = 0
    elif isinstance(agent, Firefighter):
        portrayal = {"Shape": "circle", "r": 0.5, "Color": "blue", "Layer": 1}
    return portrayal

width, height = 10, 10
density = 0.8

grid = CanvasGrid(agent_portrayal, width, height, 500, 500)

In [ ]:
server = ModularServer(
    FireModel,
    [grid],
    "Fire Suppression Model",
    {"width": width, "height": height, "density": density}
)

server.port = 8981
server.launch()